In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

covid_raw = pd.read_csv("maycovid.csv")
covid_raw

In [ ]:
covid_raw.dtypes

In [ ]:
covid_raw.shape

In [ ]:
del_col_list = ['FIPS', 'Last_Update']

covid = covid_raw.drop(del_col_list, axis=1)
covid.head()

In [ ]:
covid.columns = covid.columns.str.replace('Admin2','County')
covid.columns = covid.columns.str.replace('Long_','Long')
covid.head()

In [ ]:
print(covid.shape)
duplicate_rows_covid = covid[covid.duplicated()]
print(duplicate_rows_covid.shape)

In [ ]:
covid.columns

In [ ]:
print(covid.isnull().sum())

In [ ]:
sns.pairplot(covid)
#there appears to be more confirmed cases and more deaths as latitude increases (north)

In [ ]:
df_related = covid[['Lat','Long','Confirmed']]
sns.pairplot(df_related, kind='reg')

In [ ]:
cases = covid.sort_values(by ='Confirmed', ascending=False)
cases

In [ ]:
average_in_country = covid.groupby(["Country_Region"])[["Confirmed"]].mean()
average_in_country.sort_values(by='Confirmed', ascending = False)


In [ ]:
def find_min_max_in(col):
    """
    The function takes in a column and returns the top 5
    and bottom 5 in that column.
    
    args:
        col: string - column name
    return:
        info_df: dataframe - final 5 dataframe
    """
    
    worst = covid[col].idxmax()
    worst_df = pd.DataFrame(covid.loc[worst])
    
    best = covid[col].idxmin()
    best_df = pd.DataFrame(covid.loc[best])
    
    info_df = pd.concat([worst_df, best_df], axis=1)
    return info_df

find_min_max_in('Confirmed')

In [ ]:
  def find_min_max_in(col): 
    worst = covid[col].idxmax()
    worst_df = pd.DataFrame(covid.loc[worst])
    
    best = covid[col].idxmin()
    best_df = pd.DataFrame(covid.loc[best])
    
    info_df = pd.concat([worst_df, best_df], axis=1)
    return info_df

find_min_max_in('Deaths')


In [ ]:
average_in_lat = covid.groupby(["Lat"])[["Confirmed"]].mean()
average_in_lat.sort_values(by='Confirmed', ascending = False)

In [ ]:
covid['fatalities'] = covid['Deaths'] / covid['Confirmed']
cols = covid.columns
covid = covid[cols]
covid.sort_values(by = ['fatalities'], ascending=False)

In [ ]:
#need to figure out how to remove rows with 'inf' fatalities

In [ ]:
find_min_max_in('fatalities')

In [ ]:
average_fatalities = covid.groupby(["Country_Region"])[["fatalities"]].mean()
average_fatalities.sort_values(by = ['fatalities'], ascending=False).head()

In [ ]:
covid.corr()

In [ ]:
#List of all of the countries in the dataset 

covid.Country_Region.unique()

In [ ]:
#Add in a way to determine which territories go with which countries (and use on map?)

In [ ]:
covid[covid['Country_Region']=='Australia']['Confirmed'].mean()
#it is likely that Australia is split into territories

In [ ]:
covid[covid['Country_Region']=='United Kingdom']['Confirmed'].mean()
#It is likely that the UK is not split into territories

In [ ]:
covid.mean()

In [ ]:
covid.std()

**US Data:**

Confirmed Cases:

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].mean()

#average number of cases per US location

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].std()

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].median()

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].mode()
#This shows that due to the fact that so many US locations have no confirmed cases, the data is likely inaccurate (not sufficient testing)

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].min()

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].max()

In [ ]:
quantile1 = covid[covid['Country_Region']=='US']['Confirmed'].quantile(0.25)
quantile1

In [ ]:
quantile3 = covid[covid['Country_Region']=='US']['Confirmed'].quantile(0.75)
quantile3

In [ ]:
IQR = quantile3-quantile1
IQR

Deaths:

In [ ]:
covid[covid['Country_Region']=='US']['Deaths'].mean()

In [ ]:
covid[covid['Country_Region']=='US']['Deaths'].std()

In [ ]:
covid[covid['Country_Region']=='US']['Deaths'].median()

In [ ]:
covid[covid['Country_Region']=='US']['Deaths'].mode()

In [ ]:
covid[covid['Country_Region']=='US']['Confirmed'].max()

In [ ]:
covid[covid['Country_Region']=='US']['Deaths'].quantile(0.9)

Linear Regression and Machine Learning

In [ ]:
drop = ['fatalities']

covid = covid.drop(drop, axis=1)
covid.head()

In [ ]:
# select numerical columns
numerical_data = covid.select_dtypes(np.number).fillna(0)
numerical_data

In [ ]:
# split the numerical data into training and test sets
from sklearn.model_selection import train_test_split

target_variable = 'Confirmed'
independent_variables = numerical_data.drop(columns = target_variable).columns
independent_variables

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    numerical_data[independent_variables],
    numerical_data[target_variable],
    test_size=0.2,
    random_state=13
)

In [ ]:
print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test',  X_test.shape)
print('y_test',  y_test.shape)

In [ ]:
# create a linear regression model and fit it with the training set
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
print(model.intercept_)

print(model.coef_)

In [ ]:
# predict confirmed using test set
predictions = model.predict(X_test)

In [ ]:
X = X_test.reset_index().copy()
X['Confirmed'] = y_test.tolist()
X['prediction'] = predictions
X.head()

In [ ]:
# visualize the actual confirmed against predicted 
sns.relplot(x='Confirmed', y='prediction', data=X, kind='scatter')

In [ ]:
#visualize latitude against confirmed
sns.scatterplot(x=X["Lat"], y=X["Confirmed"], label = 'Target(truth)')
sns.scatterplot(x=X["Lat"], y=X["prediction"], label = 'Predictions')
sns.mpl.pyplot.ylabel("Confirmed cases");

# Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
from sklearn.metrics import SCORERS

In [ ]:
results_cross_validation = cross_val_score(
    estimator=model, 
    X=numerical_data[independent_variables],
    y=numerical_data[target_variable],
    scoring="neg_mean_absolute_error", 
    cv=5,
)

# 5 repetitions means 5 cv

In [ ]:
results_cross_validation

In [ ]:
model_mae = results_cross_validation.mean()
model_mae

In [ ]:
import pandas as pd
january = pd.read_csv("januarycovid.csv")
february = pd.read_csv("februarycovid.csv")
march = pd.read_csv("marchcovid.csv")
april = pd.read_csv("aprilcovid.csv")
may = pd.read_csv("maycovid.csv")

In [ ]:
january.head()

In [ ]:
february.head()

In [ ]:
march.head(10)

In [ ]:
april.head()

In [ ]:
may.tail(20)

# Time Series

In [21]:
conda install -c plotly plotly=4.8.1

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::certifi-2019.11.28-py37_0, anaconda/win-64::conda-4.8.3-py37_0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::certifi-2019.11.28-py37_0, anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::conda-4.8.3-py37_0
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::conda-4.8.3-py37_0, anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::certifi-2019.11.28-py37_0
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::certifi-2019.11.28-py37_0, defaults/win-64::conda-4.8.3-py37_0
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::conda-4.8.3-py37_0, defaults/win-64::certifi-2019.11.28-py37_0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::certifi-2019.11.28-py37_0, defaul

In [22]:
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns
sns.set()

import numpy as np 
import matplotlib 

import plotly.graph_objects as go 
import plotly.express as px

### Link to raw files for Covid-19 time series data set

In [130]:
confirmed_cases_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
death_cases_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recovered_cases_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
country_cases_link = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"

### Download datasets into respective data frames

In [131]:
confirmed_df = pd.read_csv(confirmed_cases_link)
print(confirmed_df.shape)
deaths_df = pd.read_csv(death_cases_link)
print(deaths_df.shape)
recovered_df = pd.read_csv(recovered_cases_link)
print(recovered_df.shape)
cases_country_df = pd.read_csv(country_cases_link)
print(cases_country_df.shape)

(266, 141)
(266, 141)
(253, 141)
(188, 14)


In [132]:
confirmed_df["Country/Region"].tail(20).sort_values()

252                 Botswana
247                    Burma
253                  Burundi
263                  Comoros
258                   France
246                   Kosovo
265                  Lesotho
251               MS Zaandam
256                   Malawi
255              Netherlands
261    Sao Tome and Principe
254             Sierra Leone
259              South Sudan
264               Tajikistan
250           United Kingdom
257           United Kingdom
249           United Kingdom
248           United Kingdom
260           Western Sahara
262                    Yemen
Name: Country/Region, dtype: object

In [133]:
# Looking at confirmed cases in Australia
confirmed_df[confirmed_df["Country/Region"]=="Australia"]

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,107,107,107,107,107,107,107,107,107,108
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,3092,3092,3095,3098,3104,3104,3106,3110,3110,3109
10,Northern Territory,Australia,-12.4634,130.8456,0,0,0,0,0,0,...,29,29,29,29,29,29,29,29,29,29
11,Queensland,Australia,-28.0167,153.4000,0,0,0,0,0,0,...,1058,1058,1058,1058,1059,1059,1060,1060,1061,1061
12,South Australia,Australia,-34.9285,138.6007,0,0,0,0,0,0,...,440,440,440,440,440,440,440,440,440,440
13,Tasmania,Australia,-41.4545,145.9707,0,0,0,0,0,0,...,228,228,228,228,228,228,228,228,228,228
14,Victoria,Australia,-37.8136,144.9631,0,0,0,0,1,1,...,1634,1645,1649,1653,1663,1670,1678,1681,1681,1685
15,Western Australia,Australia,-31.9505,115.8605,0,0,0,0,0,0,...,577,585,586,589,591,592,592,592,596,599


In [134]:
# There are 188 countries in this data set
confirmed_df["Country/Region"].nunique()

188

### Imputing data

In [135]:
confirmed_df = confirmed_df.replace(np.nan, '', regex=True)
deaths_df = deaths_df.replace(np.nan, '', regex=True)
recovered_df = recovered_df.replace(np.nan, '', regex=True)
cases_country_df = cases_country_df.replace(np.nan, '', regex=True)

### Viewing Columns

In [160]:
cases_country_df['Recovered'] = pd.to_numeric(cases_country_df['Recovered'], errors='coerce')

### Exploratory Analysis

In [137]:
global_data = cases_country_df.copy().drop(['Lat', 'Long_', 'Country_Region', 'Last_Update'],axis=1)
                    #,'ISO3','UID','Incident_Rate','People_Tested', 'People_Hospitalized'], axis=1)
global_summary = pd.DataFrame(global_data.sum()).transpose()

In [138]:
global_summary.style.format('{:,.0f}')

ValueError: Unknown format code 'f' for object of type 'str'

### For Chart 1: TotlalConfirmed Coronavirus Cases (Globally)

In [139]:
confirmed_ts = confirmed_df.copy().drop(['Lat', 'Long', 'Country/Region','Province/State'], axis=1)
confirmed_ts_summary = confirmed_ts.sum()

In [140]:
confirmed_ts_summary

1/22/20        555
1/23/20        654
1/24/20        941
1/25/20       1434
1/26/20       2118
            ...   
6/2/20     6378237
6/3/20     6508635
6/4/20     6632985
6/5/20     6764918
6/6/20     6891213
Length: 137, dtype: int64

In [141]:
confirmed_ts

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20
0,0,0,0,0,0,0,0,0,0,0,...,13036,13659,14525,15205,15750,16509,17267,18054,18969,19551
1,0,0,0,0,0,0,0,0,0,0,...,1076,1099,1122,1137,1143,1164,1184,1197,1212,1232
2,0,0,0,0,0,0,0,0,0,0,...,8997,9134,9267,9394,9513,9626,9733,9831,9935,10050
3,0,0,0,0,0,0,0,0,0,0,...,763,764,764,764,765,844,851,852,852,852
4,0,0,0,0,0,0,0,0,0,0,...,74,81,84,86,86,86,86,86,86,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0,0,0,0,0,0,0,0,0,0,...,458,463,479,483,484,484,484,485,499,499
262,0,0,0,0,0,0,0,0,0,0,...,278,283,310,323,354,399,419,453,469,482
263,0,0,0,0,0,0,0,0,0,0,...,87,87,106,106,106,132,132,132,132,141
264,0,0,0,0,0,0,0,0,0,0,...,3563,3686,3807,3930,4013,4100,4191,4289,4370,4453


In [142]:
fig_1 = go.Figure(data=go.Scatter(x=confirmed_ts_summary.index, y=confirmed_ts_summary.values,
                       mode='lines+markers'))
fig_1.update_layout(title='Total Coronavirus Confirmed Cases (Globally)',
                   yaxis_title='Confirmed Cases', xaxis_tickangle= 315)
fig_1.show()

### Defining a template plot function and color array

In [143]:
# Initializing Color array to be used across the analysis
color_arr = px.colors.qualitative.Dark24

In [144]:
def draw_plot(ts_array, ts_label, title, colors, mode_size, line_size, 
             x_axis_title, y_axis_title, tickangle = 0, yaxis_type='',
             additional_annotations=[]):
    # initialize figure
    fig = go.Figure()
    # add all traces
    for index, ts in enumerate(ts_array):
        fig.add_trace(go.Scatter(x=ts.index,
                                y=ts.values,
                                name=ts_label[index],
                                line=dict(color=colors[index],width=line_size[index]),connectgaps=True,))
        
    # base x axis prop
    x_axis_dict = dict(showline=True,
                      showgrid=True,
                      showticklabels=True,
                      linecolor='rgb(204,204,204)',
                      linewidth=2,
                      ticks='outside',
                      tickfont=dict(family='Arial',size=12, color='rgb(82,82,82)',))
    
    # setting x_axis params
    if x_axis_title:
        x_axis_dict['title'] = x_axis_title
        
    if tickangle > 0:
        x_axis_dict['tickangle']=tickangle

    # base y_axis
    y_axis_dict = dict(showline=True,
                      showgrid=True,
                      showticklabels=True,
                      linecolor='rgb(204,204,204)',
                      linewidth=2)
    
    # setting y_axis params
    if yaxis_type != '':
        y_axis_dict['type']=yaxis_type
        
    if y_axis_title:
        y_axis_dict['title'] = y_axis_title
        
#updating the layout
    fig.update_layout(xaxis=x_axis_dict,
                     yaxis=y_axis_dict,
                     autosize=True,
                     margin=dict(autoexpand=True, l=100,r=20,t=110,),
                     showlegend=True)
    
    # base annotations for any graph
    annotations = []
    # Title
    annotations.append(dict(xref='paper', yref='paper', x=0.0, y= 1.05, xanchor='left',
                           yanchor='bottom',text=title,
                            font=dict(family='Arial',size=16,color='rgb(37,37,37)'),showarrow=False))
    # adding annotations in params
    if len(additional_annotations) > 0:
        annotations.append(additional_annotations)
        
    # updating the layout
    fig.update_layout(annotations=annotations)
    
    return fig

### For Chart 2: Covid-19 Case Status

In [145]:
confirmed_agg_ts = confirmed_df.copy().drop(['Lat','Long','Country/Region',
                                            'Province/State'],axis=1).sum()
death_agg_ts = deaths_df.copy().drop(['Lat','Long','Country/Region',
                                            'Province/State'],axis=1).sum()
recovered_agg_ts = recovered_df.copy().drop(['Lat','Long','Country/Region',
                                            'Province/State'],axis=1).sum()

# There is no timeseries data for Active cases, therefore it needs to be engineered
active_agg_ts = pd.Series(
    data=np.array(
        [(x1 - x2) - x3 for (x1, x2, x3) in zip(confirmed_agg_ts.values, death_agg_ts.values, 
                                             recovered_agg_ts.values)]),
    index=confirmed_agg_ts.index)

In [146]:
death_agg_ts

1/22/20        17
1/23/20        18
1/24/20        26
1/25/20        42
1/26/20        56
            ...  
6/2/20     380249
6/3/20     385947
6/4/20     391136
6/5/20     395880
6/6/20     399718
Length: 137, dtype: int64

In [148]:
ts_array = [confirmed_agg_ts, active_agg_ts, recovered_agg_ts, death_agg_ts]
labels = ['Confirmed', 'Active', 'Recovered', 'Deaths']
colors = [color_arr[5], color_arr[0], color_arr[2], color_arr[3]]
mode_size = [8,8,12,8]
line_size = [2,2,4,2]

fig_2 = draw_plot(ts_array = ts_array,
                 ts_label = labels,
                 title = 'Covid-19 Case Status (22nd Jan to 7th June 2020)',
                 colors = colors, mode_size = mode_size,
                 line_size = line_size,
                 x_axis_title = 'Date',
                 y_axis_title = 'Case Count',
                 tickangle=315,
                 yaxis_type='', additional_annotations=[])
fig_2.show()

### For Country Level Drill Down

In [170]:
import pandas.io.formats.style

In [181]:
# Why is this not formatting?
pd.options.display.float_format = '{:,.2f}'.format
cases_copy = cases_country_df.copy().drop(
    ['Lat', 'Long_','Last_Update','People_Tested','People_Hospitalized'],axis = 1).sort_values('Recovered', ascending=False).reset_index(drop=True).style.bar(
    align='left',width=98,color='#d65f5f')
cases_copy

### For Chart 3: "Covid-19 Case Trend in United States"


In [ ]:
confirmed_us_ts = confirmed_df[confirmed_df['Country/Region']=='India']
confirmed_us_ts = confirmed_